# Import models

In [ ]:
import modal

MODEL_DIR = "/model"
MODEL_NAME = "deepseek-ai/DeepSeek-Coder-V2-Lite-Instruct"

: 

In [ ]:
image = (
    modal.Image.debian_slim(python_version="3.11")
    .pip_install(
        "torch==2.5.1",
        "transformers==4.47.1",
        "hf-transfer==0.1.8",
        "vllm"
    )
    # Use the barebones `hf-transfer` package for maximum download speeds. No progress bar, but expect 700MB/s.
    .env({"HF_HUB_ENABLE_HF_TRANSFER": "1", "HF_HUB_CACHE": MODEL_DIR})
)

: 

In [5]:
model_cache = modal.Volume.from_name("hf-hub-cache", create_if_missing=True)
app = modal.App(
    "CodeRoaster",
    image=image,
    volumes={MODEL_DIR: model_cache},
)

In [6]:
@app.function()
def download_model():
    from huggingface_hub import snapshot_download
    from transformers.utils import move_cache

    snapshot_download(
        MODEL_NAME,
        ignore_patterns=["*.pt", "*.bin"],  # Using safetensors
    )
    move_cache()

In [ ]:
with modal.enable_output():
    with app.run():
        download_model.remote()

# Test api endpoint

In [ ]:
test_code = """
class TrieNode():
    def __init__(self):
        self.children = {}
        self.eow = False

class Solution:
    def longestWord(self, words: List[str]) -> str:
        trie_root = TrieNode()

        # add all words to trie
        for word in words:
            curr_node = trie_root
            for char in word:
                if char not in curr_node.children:
                    curr_node.children[char] = TrieNode()
                curr_node = curr_node.children[char]
            curr_node.eow = True

        res = ""

        queue = deque([(trie_root, "")]) # node, list of char

        while queue:
            res = queue[0][1]

            for _ in range(len(queue)):
                curr_node, curr_string = queue.popleft()
                res = min(res, curr_string)

                for next_char in curr_node.children:
                    if curr_node.children[next_char].eow == True:
                        queue.append((curr_node.children[next_char], curr_string + next_char))
        return res
    """

: 

In [ ]:
import requests
import yaml

# Define the URL of the endpoint
url = "https://notshowingtoyou.run"  # Replace with the actual URL if hosted elsewhere

# Define the payload
payload = {
    "code": test_code
}

# Send the POST request
response = requests.post(url, json=payload)

# Print the response
if response.status_code == 200:
    print("Response:", response.json())
else:
    print("Error:", response.status_code, response.text)

Response: You are a brutally honest, sarcastic, and witty senior software engineer. Your job is to roast people's LeetCode code solutions.

You're given a code snippet. Your job is to roast it in a funny, clever, and mildly insulting way — like a snarky code reviewer on a bad day.

Be playful, creative, and don't hold back. Make comments about bad logic, unnecessary complexity, cringe variable names, poor edge case handling, or anything else that deserves it.

Here's the code to roast:


class TrieNode():
    def __init__(self):
        self.children = {}
        self.eow = False

class Solution:
    def longestWord(self, words: List[str]) -> str:
        trie_root = TrieNode()

        # add all words to trie
        for word in words:
            curr_node = trie_root
            for char in word:
                if char not in curr_node.children:
                    curr_node.children[char] = TrieNode()
                curr_node = curr_node.children[char]
            curr_node.eow =